# Извлечение данных

На этом занятии мы расследуем загадочное убийство с помощью SQL-запросов!

In [1]:
!git clone https://github.com/NUKnightLab/sql-mysteries.git

Cloning into 'sql-mysteries'...
fatal: unable to access 'https://github.com/NUKnightLab/sql-mysteries.git/': Recv failure: Connection reset by peer


[Задание](sql-mysteries/prompt_beginner.pdf)

Схема базы данных:

<img src = "schema.png"/>

## Подключение и работа с базой данных на примере SQLite

In [2]:
import sqlite3
import pandas as pd

In [3]:
connection = sqlite3.connect('sql-murder-mystery.db')

In [4]:
with connection:
    message_list = connection.execute(
        """
        SELECT * FROM crime_scene_report
        ;
        """
    )

In [5]:
for n, item in enumerate(message_list):
    print(item)
    if n > 5:
        break

(20180115, 'robbery', 'A Man Dressed as Spider-Man Is on a Robbery Spree', 'NYC')
(20180115, 'murder', 'Life? Dont talk to me about life.', 'Albany')
(20180115, 'murder', 'Mama, I killed a man, put a gun against his head...', 'Reno')
(20180215, 'murder', 'REDACTED REDACTED REDACTED', 'SQL City')
(20180215, 'murder', 'Someone killed the guard! He took an arrow to the knee!', 'SQL City')
(20180115, 'theft', 'Big Bully stole my lunch money!', 'Chicago')
(20180115, 'fraud', 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do\n            eiusmod tempor incididunt ut labore et dolore magna aliqua.', 'Seattle')


Для подключения к другим серверам можно воспользоваться модулем sqlalchemy:

In [ ]:
# from sqlalchemy import create_engine
# connection = create_engine("postgresql:///?User=username&;Password=password&Database=database_name&Server=127.0.0.1&Port=5432")

### Работа с подключением через Pandas

In [ ]:
pd.read_sql(
        """
        SELECT * FROM crime_scene_report
        ;
        """,
        connection)

### Условия и сортировка в запросе SELECT

In [ ]:
df = pd.read_sql(
        """
        SELECT * FROM crime_scene_report
        WHERE type = "murder"
        AND date = 20180115
        AND city = "SQL City"
        ;
        """,
        connection)

In [ ]:
df

In [ ]:
df['description'].loc[0]

In [ ]:
df = pd.read_sql(
        """
        SELECT * FROM person
        ;
        """,
        connection)

In [ ]:
df.head()

In [ ]:
pd.read_sql(
        """
        SELECT * FROM person
        WHERE address_street_name = "Northwestern Dr"
        ORDER BY address_number DESC
        ;
        """,
        connection).head()

#### Самостоятельная работа

Найдите второго свидетеля.

In [ ]:
## Ваш код ниже:


### Объединение таблиц

In [ ]:
pd.read_sql(
        """
        SELECT * FROM 
        person INNER JOIN interview ON person.id = interview.person_id
        ;
        """,
        connection).head()

In [ ]:
df = pd.read_sql(
        """
        SELECT * FROM 
        person INNER JOIN interview ON person.id = interview.person_id
        WHERE id = 14887 or id = 16371
        ;
        """,
        connection)

In [ ]:
df

In [ ]:
df['transcript'].loc[0]

In [ ]:
df['transcript'].loc[1]

In [ ]:
pd.read_sql(
        """
        SELECT * FROM
        get_fit_now_member INNER JOIN get_fit_now_check_in ON get_fit_now_member.id = get_fit_now_check_in.membership_id
        WHERE id LIKE "48Z%"
        AND check_in_date = 20180109
        ;
        """,
        connection)

In [ ]:
pd.read_sql(
        """
        SELECT * FROM
        get_fit_now_member INNER JOIN get_fit_now_check_in ON get_fit_now_member.id = get_fit_now_check_in.membership_id
        INNER JOIN person ON get_fit_now_member.person_id = person.id
        INNER JOIN drivers_license ON person.license_id = drivers_license.id
        WHERE get_fit_now_member.id LIKE "48Z%"
        AND check_in_date = 20180109
        ;
        """,
        connection)

In [ ]:
with connection:
    message_list = connection.execute(
        """
        INSERT INTO solution VALUES (1, 'Jeremy Bowers');
        """
    )   

In [ ]:
pd.read_sql(
        """
        SELECT value FROM solution;
        """,
        connection)['value'][0]

In [ ]:
df = pd.read_sql(
        """
        SELECT * FROM 
        person INNER JOIN interview ON person.id = interview.person_id
        WHERE id = 67318
        ;
        """,
        connection)

In [ ]:
df['transcript'].loc[0]

In [ ]:
pd.read_sql(
        """
        SELECT AVG(annual_income), MAX(annual_income) FROM 
        person
        LEFT JOIN income ON person.ssn = income.ssn
        ;
        """,
        connection)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_sql(
        """
        SELECT * FROM 
        income
        ;
        """,
        connection)

In [ ]:
df.describe()

In [ ]:
translations = {
    'annual_income' : 'Годовой доход',
}

def visualize(column, xlim=None):
    xlabel = translations.get(column.name, column.name)
    fig, axes = plt.subplots(1, 2, figsize=(16,6))
    sns.histplot(column, ax=axes[0], bins='fd', kde=False)
    axes[0].set(title='Гистограмма', xlabel=xlabel, ylabel='Наблюдений, шт.', xlim=xlim);
    sns.violinplot(column, ax=axes[1], orient='h')
    axes[1].set(title='Плотность и размах', xlabel=xlabel, xlim=xlim);
    
visualize(df['annual_income'])

In [ ]:
pd.read_sql(
        """
        SELECT DISTINCT car_model FROM 
        drivers_license
        ;
        """,
        connection)

### Группировка

In [ ]:
pd.read_sql(
        """
        SELECT type, count(*) as count FROM crime_scene_report
        GROUP BY type
        ;
        """,
        connection)

#### Самостоятельная работа

Сделайте аналог value_counts() pandas по моделям автомашин с помощью группировки:

In [ ]:
pd.read_sql(
        """
        SELECT * FROM 
        drivers_license
        ;
        """,
        connection)['car_model'].value_counts()

In [ ]:
# Ваш код:


### Категоризация с помощью CASE

In [ ]:
pd.read_sql(
        """
        SELECT
        CASE
            WHEN annual_income < 30500 THEN 'Poor'
            WHEN annual_income > 30500 AND annual_income < 50800 THEN 'Average' 
            WHEN annual_income > 50800 AND annual_income < 70800 THEN 'Decent'
            WHEN annual_income > 70800 THEN 'Rich'
            ELSE 'Unknown'
        END
        as income_level, 
        count(person.id)
        FROM 
        person LEFT JOIN drivers_license on drivers_license.id = person.license_id
        LEFT JOIN income ON person.ssn = income.ssn
        GROUP BY income_level
        ;
        """,
        connection)

### Фильтрация после группировки

In [ ]:
pd.read_sql(
        """
        SELECT * FROM facebook_event_checkin
        ;
        """,
        connection)

In [ ]:
pd.read_sql(
        """
        SELECT person.id, name, COUNT(*) as count FROM 
        person LEFT JOIN facebook_event_checkin ON person.id = facebook_event_checkin.person_id
        LEFT JOIN income ON person.ssn = income.ssn
        LEFT JOIN drivers_license ON person.license_id = drivers_license.id
        WHERE event_name LIKE "SQL Symphony Concert"
        AND date LIKE "201712%"
        GROUP BY person.id, name
        HAVING count = 3
        ;
        """,
        connection)

In [ ]:
pd.read_sql(
        """
        SELECT * FROM 
        person
        LEFT JOIN income ON person.ssn = income.ssn
        LEFT JOIN drivers_license ON person.license_id = drivers_license.id
        WHERE person.id = 99716
        ;
        """,
        connection)

In [ ]:
with connection:
    message_list = connection.execute(
        """
        INSERT INTO solution VALUES (1, 'Miranda Priestly');
        """
    )   

In [ ]:
pd.read_sql(
        """
        SELECT value FROM solution;
        """,
        connection)['value'][0]

# Домашнее задание

## Easy

Рассмотрите таблицу `crime_scene_report` и выведите уникальные категории преступлений (столбец `type`).

In [6]:
# Ваш код ниже:
pd.read_sql(
    """
    SELECT DISTINCT type FROM 
    crime_scene_report
    ;
    """,
    connection)

,type
0,robbery
1,murder
2,theft
3,fraud
4,arson
5,bribery
6,assault
7,smuggling
8,blackmail


## Normal

Оцените, зависит ли средний доход человека от цвета глаз (для этого вам понадобится объединить три таблицы.)

In [28]:
# Ваш код ниже:
pd.read_sql(
    """
    SELECT eye_color, avg(annual_income)
    FROM person as p
        inner join income as i on p.ssn = i.ssn
        inner join drivers_license as dl on p.license_id = dl.id
    GROUP BY eye_color
    ;
    """,
    connection
)

,eye_color,avg(annual_income)
0,amber,51608.310811
1,black,53952.546917
2,blue,53397.780679
3,brown,54065.119363
4,green,53239.680426


## Hard

Прикиньте средний доход человека в зависимости от роста. С помощью SQL-запроса выведите значения роста, при которых люди в среднем получают больше, чем те, кто на 1-3 дюйма ниже их.

In [50]:
# Ваш код ниже:
pd.read_sql(
    """
    SELECT * FROM (
        SELECT
            dl.height,
            avg(i.annual_income),
            avg(i.annual_income) Filter(where $height > height)
        FROM person as p
            inner join income as i on p.ssn = i.ssn
            inner join drivers_license as dl on p.license_id = dl.id
        GROUP BY dl.height
    )
    ;
    """,
    connection
)

DatabaseError: Execution failed on sql '
    SELECT * FROM (
        SELECT
            dl.height,
            avg(i.annual_income),
            avg(i.annual_income) Filter(where $height > height)
        FROM person as p
            inner join income as i on p.ssn = i.ssn
            inner join drivers_license as dl on p.license_id = dl.id
        GROUP BY dl.height
    )
    ;
    ': Incorrect number of bindings supplied. The current statement uses 1, and there are 0 supplied.